## 라이브러리 및 딕셔너리를 선언

In [ ]:
!pip install transformers

In [ ]:
#!pip install smplx==0.1.13

In [92]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import cv2
import numpy as np
import timm
import os
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [93]:
fish_category = {'Olive_flounder' : 0,
                 'Korea_rockfish' : 1,
                 'Red_seabream' : 2,
                 'Black_porgy' : 3,
                 'Rock_bream' : 4,
                 'Croaker' : 5,
                 'Argyrosomus_japonicus' : 6,
                 'Starry_flounder' : 7,
                 'Longtooth_grouper' : 8,
                 'Convict_grouper' : 9,
                 'Japanese_amberjack' : 10,
                 'Yellowtail_amberjack' : 11,
                }

### cuda 연결

In [94]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'Currently using {device}...')

Currently using cuda...


### model pth 가져오기

In [95]:
class TempModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.resnet = timm.create_model('resnet50', pretrained = True, num_classes = num_classes, drop_rate=0.5)

    def forward(self, x):
        x = self.resnet(x)
        return x
        
file_path = './fish_Resnet50_best_epoch24_0.9012.pth'
model = TempModel(num_classes =12)
model.to(device)
model.load_state_dict(torch.load(file_path))
model.eval()

TempModel(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        

### 계산을 위해 Feature map 과 class weight를 가져오자 

##### hook을 이용

In [96]:
grad = None
class_weight = None

#feature_module = "layer4" #resnet의 마지막 layer
#target_layer_names = 2 #feature_module의 마지막 BottleNeck의 이름



# 마지막 Conv의 gradient

def hook_4_grad(model,grad_input,grad_output):
    global grad
    print("1111")
    grad = model.grad#.clone().detach()

def hook_4_clf_w(model,input,output):
    global class_weight
    class_weight = model.weight

model.resnet.layer4[2].register_backward_hook(hook_4_grad)
#model.resnet.fc.register_forward_hook(hook_4_clf_w)
print(grad)



None


### 데이터 전처리

In [97]:
import albumentations as A
import cv2
from albumentations.pytorch.transforms import ToTensorV2

class transforms_1:
    def __init__(self, resize) -> None:
        self.transforms = A.Compose([
            A.Resize(*resize, cv2.INTER_LINEAR),
            A.Normalize(),
            ToTensorV2(),
        ])
    
    def __call__(self, image):
        return self.transforms(image=image)

transform=A.Compose([
        A.Resize(384, 384),
        A.Normalize(),
        ToTensorV2(),
    ])

trans_resize = A.Compose([
    A.Resize(384,384)
])

In [98]:
current_path = os.getcwd()
dataset_dir = os.path.join(current_path,"test_data")

data_list = os.listdir(dataset_dir)
data_list

['방어.jpg']

In [99]:
image_name = data_list[0]

#추후 비교를 위해 따로 저장
origin_image = np.array(cv2.imread(os.path.join(dataset_dir,image_name)))
origin_image = trans_resize(image=origin_image)['image']

#학습을 위한 데이터
image = transform(image=origin_image)['image']
image = image.unsqueeze(0)
image = image.to(device, dtype=torch.float32)


In [100]:
output = model(image)
preds = torch.argmax(output, dim=-1)
# tmp = grad.squeeze(axis = 0)
# print(tmp.shape)
# print(class_weight.shape)
# label_weight = class_weight[preds[0]]
# print(label_weight.shape)
# label_weight = label_weight.unsqueeze(-1).unsqueeze(-1)
# print(label_weight.shape)
criterion = nn.CrossEntropyLoss()
loss = criterion(output, preds)
loss.backward()

1111


ModuleAttributeError: 'Bottleneck' object has no attribute 'grad'

In [ ]:
cam = torch.squeeze(tmp)* label_weight
cam = torch.sum(cam,axis = 0)
cam = cam.detach().cpu().numpy()

In [ ]:
#origin = origin_img.detach().cpu().numpy()
from mpl_toolkits.axes_grid1 import make_axes_locatable

final_cam = cv2.resize(cam, dsize=(384, 384), interpolation= cv2.INTER_CUBIC)
img_rgb = cv2.cvtColor(origin_image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(20,40))

plt.subplot(1,3,1)
plt.title("CAM + Image")

plt.imshow(img_rgb)
plt.imshow(final_cam,cmap = 'jet' ,alpha=0.6)

plt.subplot(1,3,2)
plt.title("Origin")
plt.imshow(img_rgb)

ax = plt.subplot(1,3,3)
plt.title("CAM",alpha=0.4)
im =plt.imshow(final_cam, cmap = 'jet')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size = "5%", pad = 0.05)
plt.colorbar(im, cax= cax)
print("정답 : ", image_name)
keys = fish_category.keys()
key_lst = list(keys)
print("예측 : ",key_lst[preds[0]])

### 연속 실행

In [ ]:
current_path = os.getcwd()
dataset_dir = os.path.join(current_path,"test_data")

data_list = os.listdir(dataset_dir)


def CAM (image_name):
    #추후 비교를 위해 따로 저장
    origin_image = np.array(cv2.imread(os.path.join(dataset_dir,image_name)))
    origin_image = trans_resize(image=origin_image)['image']

    #학습을 위한 데이터
    image = transform(image=origin_image)['image']
    image = image.unsqueeze(0)
    image = image.to(device, dtype=torch.float32)


    #inference 진행
    output = model(image)
    preds = torch.argmax(output, dim=-1)

    #feature map 뽑아내기
    tmp = activation.squeeze(axis = 0)

    #class weight 뽑아내기
    label_weight = class_weight[preds[0]]

    #연산을 위한 차원 변환
    label_weight = label_weight.unsqueeze(-1).unsqueeze(-1)

    #cam 만들기
    cam = torch.squeeze(tmp)* label_weight
    cam = torch.sum(cam,axis = 0)
    cam = cam.detach().cpu().numpy()

    # 사진을 겹쳐보기 위한 resize 해주기
    final_cam = cv2.resize(cam, dsize=(384, 384), interpolation= cv2.INTER_CUBIC)
    img_rgb = cv2.cvtColor(origin_image, cv2.COLOR_BGR2RGB)


    #시각화
    plt.figure(figsize=(20,40))

    plt.subplot(1,3,1)
    plt.title("CAM + Image")

    plt.imshow(img_rgb)
    plt.imshow(final_cam,cmap = 'jet' ,alpha=0.6)

    plt.subplot(1,3,2)
    plt.title("Origin")
    plt.imshow(img_rgb)

    ax = plt.subplot(1,3,3)
    plt.title("CAM")
    im =plt.imshow(final_cam, cmap = 'jet')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size = "5%", pad = 0.05)
    plt.colorbar(im, cax= cax)
    print("파일이름 : ", image_name)
    keys = fish_category.keys()
    key_lst = list(keys)
    print("예측 : ",str(key_lst[preds[0]]))

## 아래의 코드를 연속 실행해주자

In [ ]:
#초기화
index = 0

In [ ]:
try:
    CAM(data_list[index])
    index +=1
except IndexError:
    print("Finish")